# Predict cell fates using the CNN-RNN approach

To predict cell fates using our approach, we need the `caffe` framework (to extract features) as well as `Theano` (to do the acutal prediction). To install caffe, first clone our fork from [here](https://github.com/flophys/caffe) (providing full support for concatenation layer).  Then follow stanard `caffe`  installation instructions that can be found [here](http://caffe.berkeleyvision.org/installation.html). `Theano` can be installed following the instructions detailed [here](http://deeplearning.net/software/theano/install.html). We recommend using the Anaconda distribution for an easy installation process of both `Theano` and `caffe`.
Predictions are not computationally expensive and can be performed using CPU on a standard laptop (no cuda/GPU requiured to run this notebook).

In [ ]:
import os
import glob
import scipy as SP
import sys

caffe_root = '/Users/flo/software/caffe/'  #root directory of the caffe installation
sys.path.append(caffe_root + 'python')
sys.path.append('./')

import caffe
import py.extract_features as extr#import mat2dict, extract
from py.rnn import RNN

Having loaded the required packages, we load images and displacement features from matlab file into dictionary.

In [30]:
reload(extr)
mat_files = glob.glob('../processes_data/cell*.mat') #path to the cell for which we would like to obtain a fate prediction
cellIDs = [mat_file.split('/')[2].split('.')[0] for mat_file in mat_files] #get cellIDs from file names

nCells = len(mat_files)

ims = extr.mat2dict(mat_files) #load images and displacement features from matlab file into dictionary



Next, we extract the CNN-features for a given cell, using a pre-trained network we provide.



In [32]:
num_iter = 45000
film_tr = "120602PH5"

PRETRAINED = './models/cnn_models/'+film_tr+'/trained_CNN.caffemodel' #path to the pretrained model
MODEL_FILE = os.path.join(caffe_root,'examples/single_cells/CNN_deploy.prototxt') #model file specifying the architecture of the pre-trained model

feat_list = []
labels = []
predsCNN = []
for i in range(nCells):
    res = extr.extract(pretrained=PRETRAINED, model_file=MODEL_FILE, input_image_test=ims[i]['im'], 
                  displacement_test=ims[i]['mov'])#extract the CNN features
    feat_list.append(res['feats'])
    labels.append(ims[i]['label'][0].astype('int'))
    predsCNN.append(res['pred_all'].mean())


We can next pass these image-patch specific features into the RNN-part of the model and get a cell-specific prediction based on CNN features. To this end, we first populate the RNN with the pre-trained parameters. Next we pass t

In [33]:
fn = './models/rnn_models/120602PH5/trained_modelRNN.pkl'#pre-trained model

#load model parameters
f = open(fn, 'rb')
[structure, weights] = pickle.load(f)

#populate model
model = RNN(structure[0], structure[1], structure[2], 'dblstm')
model.set_theta(weights)

#run prediction
pred = model.predict(feat_list) #predict from CNN-based features

#fate predictions
zip(cellIDs, pred.ravel(), SP.vstack(predsCNN).ravel(), SP.vstack(labels).ravel().astype('str'))

OK


[('cell_1', 0.11399163, 0.17840698287398513, '3'),
 ('cell_10', 0.10822291, 0.15677303846267113, '1'),
 ('cell_12', 0.53996056, 0.43691967104474994, '2'),
 ('cell_164', 0.93448567, 0.48321249830845425, '-1'),
 ('cell_193', 0.96923244, 0.64816473803822272, '-1'),
 ('cell_2', 0.17836502, 0.24097076373953469, '3'),
 ('cell_20', 0.10698476, 0.12209481045533634, '1'),
 ('cell_24', 0.90018779, 0.45790841470098298, '2'),
 ('cell_3', 0.32182574, 0.2714828827025732, '3'),
 ('cell_41', 0.11717759, 0.26383823449939292, '-1'),
 ('cell_48', 0.39450011, 0.36484276000510729, '2'),
 ('cell_5', 0.34744716, 0.37612379915513022, '3'),
 ('cell_6', 0.47294822, 0.3743198669469826, '2'),
 ('cell_82', 0.13798042, 0.2585519005878622, '-1'),
 ('cell_96', 0.69771969, 0.41234238799915607, '2')]